In [1]:
import pandas as pd

In [8]:
df_valor = pd.read_parquet('/home/filipe/scripts/psdb-projetos/dashStreamlit/datasets/2022/contas2022.parquet')
df_valor = df_valor[['id', 'totalPartidos']]
df_valor['totalPartidos'] = df_valor['totalPartidos'].apply(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

In [9]:
df_valor.head(5)

,id,totalPartidos
0,10001724875,"107.957,00"
1,10001613550,"1.800.000,00"
2,280001600179,"0,00"
3,10001595335,"0,00"
4,10001639519,"2.980.000,00"


In [2]:
df_municipio = pd.read_parquet('/home/filipe/scripts/psdb-projetos/dashStreamlit/datasets/diversos/municipio.parquet')
df_municipio = df_municipio[['id','uf', 'nome', 'capital', 'no_regiao_brasil']].rename(columns={'nome' : 'municipio', 'id': 'id_municipio'})
df_municipio = df_municipio[~df_municipio['uf'].str.startswith('ZZ')] # Remove linhas com UF ZZ

In [3]:
df_municipio.head(2)

,id_municipio,uf,municipio,capital,no_regiao_brasil
0,1120,AC,ACRELÂNDIA,False,Norte
1,1570,AC,ASSIS BRASIL,False,Norte


In [4]:
df_votacao = pd.read_parquet('/home/filipe/scripts/psdb-projetos/dashStreamlit/datasets/2022/votomunicipio2022.parquet')
df_votacao = df_votacao[df_votacao['turno'] == 1]
df_votacao = df_votacao[['nu_titulo_eleitor', 'municipio', 'votos']].rename(columns={'municipio': 'id_municipio'})

In [5]:
df_votos_eleicao = pd.merge(
    df_votacao, 
    df_municipio,
    on='id_municipio',
    how='inner')

In [18]:
df_votos_eleicao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8367313 entries, 0 to 8367312
Data columns (total 7 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   nu_titulo_eleitor  object
 1   id_municipio       int64 
 2   votos              int64 
 3   uf                 object
 4   municipio          object
 5   capital            bool  
 6   no_regiao_brasil   object
dtypes: bool(1), int64(2), object(4)
memory usage: 391.0+ MB


In [6]:
df_votos_eleicao.head(5)

,nu_titulo_eleitor,id_municipio,votos,uf,municipio,capital,no_regiao_brasil
0,000780342402,1007,1670,AC,BUJARI,False,Norte
1,002324422437,1007,946,AC,BUJARI,False,Norte
2,000825962445,1007,264,AC,BUJARI,False,Norte
3,004409242488,1007,8,AC,BUJARI,False,Norte
4,000259632429,1007,80,AC,BUJARI,False,Norte


In [ ]:
votos = (
    df_votos_eleicao.groupby(['nu_titulo_eleitor'])['votos']
    .sum()
    .reset_index()
)

In [2]:
df_fichas = pd.read_parquet('/home/filipe/scripts/psdb-projetos/dashStreamlit/datasets/2022/fichas2022.parquet')
df_fichas = df_fichas[df_fichas['ds_sit_urna'] == 'Consta da urna']
df_fichas = df_fichas[df_fichas['sit_partido'].isin (['Deferido', 'Deferido com recurso'])]
df_fichas = df_fichas[df_fichas['sit_cand'].isin (['Deferido', 'Deferido com recurso'])]
df_fichas['ds_eleicao'] = df_fichas['ds_eleicao'].replace({
    'Eleito por média': 'Eleito',
    'Eleito por QP': 'Eleito',
    'Suplente' : 'Não eleito',
    'Concorrendo' : 'Não eleito',
})
df_fichas['nm_cargo'] = df_fichas['nm_cargo'].replace({'Deputado Distrital': 'Deputado Estadual'})

df_fichas.drop(columns=['id_cand','ds_sit_urna', 'sit_partido', 'sit_cand', 'nu_cpf',
                        'st_reeleicao', 'dt_atualizacao', 'nm_partido', 'nu_partido',
                        'ds_coligacao', 'nu_urna', 'cd_sit_candidato', 'ds_sit_candidato', 
                        'ds_totalizacao', 'cd_cargo', 'ds_email', 'nu_cnpj'], inplace=True)

In [3]:
df_fichas.head(2)

,uf_cand,nm_municipio,nm_cargo,nm_cand,sg_partido,ds_eleicao,ds_raca,ds_genero,nu_titulo_eleitor,dt_nascimento,ds_composicao,li_foto
0,AC,ACRE,1º Suplente,ELISABETH MEDEIROS DE MORAES,PL,Não eleito,BRANCA,FEM.,000746382402,1955-11-11,REPUBLICANOS / PL / PRTB / PSC / MDB,https://datarepo.psdb.org.br/fotos/10001643707...
1,AC,ACRE,Vice-governador,JANE DE SOUZA ROSAS,PSOL,Não eleito,BRANCA,FEM.,002460232488,1975-10-29,**,https://datarepo.psdb.org.br/fotos/10001595335...


In [18]:
top_5_siglas1 = (
    df_fichas[
        (df_fichas['nm_cargo'] == 'Deputado Federal') & 
        (df_fichas['ds_eleicao'] == 'Eleito')
    ]
    .groupby('sg_partido')
    .size()
    .nlargest(5)
    .index
    .tolist()
)

top_5_siglas1

['PL', 'PT', 'UNIÃO', 'PP', 'MDB']

In [ ]:
filtro = 'Deputado Federal'  # Example filter, can be changed to 'Deputado Estadual' or others
df_eleitos = df_fichas[(df_fichas['nm_cargo'] == filtro) & (df_fichas['ds_eleicao'] == 'Eleito')]
eleitos_partido = df_eleitos.groupby(['sg_partido']).size().reset_index(name='Eleitos por partido')
top_5_siglas = eleitos_partido.nlargest(5, 'Eleitos por partido')['sg_partido'].tolist()

In [17]:
top_5_siglas = eleitos_partido.nlargest(5, 'Eleitos por partido')['sg_partido'].tolist()
top_5_siglas

['PL', 'PT', 'UNIÃO', 'PP', 'MDB']

In [ ]:
df_eleitos.head(2)

,uf_cand,nm_municipio,nm_cargo,nm_cand,sg_partido,ds_eleicao,ds_raca,ds_genero,nu_titulo_eleitor,dt_nascimento,ds_composicao,li_foto
74,AC,ACRE,Deputado Federal,ULYSSES FREITAS PEREIRA DE ARAUJO,UNIÃO,Eleito,PRETA,MASC.,002406702453,1972-10-19,**,https://datarepo.psdb.org.br/fotos/10001612487...
79,AC,ACRE,Deputado Federal,EDUARDO OVIDIO BORGES DE VELLOSO VIANNA,UNIÃO,Eleito,BRANCA,MASC.,003036512402,1976-07-08,**,https://datarepo.psdb.org.br/fotos/10001612484...


Fim

In [8]:
df_fichas.head(2)

,uf_cand,nm_municipio,nm_cargo,nm_cand,sg_partido,ds_eleicao,ds_raca,ds_genero,nu_titulo_eleitor,dt_nascimento,ds_composicao,li_foto
0,AC,ACRE,1º Suplente,ELISABETH MEDEIROS DE MORAES,PL,Não eleito,BRANCA,FEM.,000746382402,1955-11-11,REPUBLICANOS / PL / PRTB / PSC / MDB,https://datarepo.psdb.org.br/fotos/10001643707...
1,AC,ACRE,Vice-governador,JANE DE SOUZA ROSAS,PSOL,Não eleito,BRANCA,FEM.,002460232488,1975-10-29,**,https://datarepo.psdb.org.br/fotos/10001595335...


In [9]:
df_fichas_com_votos = pd.merge(
    df_fichas,
    df_votos_eleicao,    
    left_on='nu_titulo_eleitor',
    right_on='nu_titulo_eleitor',
    how='right')

In [10]:
df_votos_eleicao.shape[0], df_fichas_com_votos.shape[0]

(8367313, 8367313)

In [11]:
df_votos_eleicao.head(2)

,nu_titulo_eleitor,id_municipio,votos,uf,municipio,capital,no_regiao_brasil
0,000780342402,1007,1670,AC,BUJARI,False,Norte
1,002324422437,1007,946,AC,BUJARI,False,Norte


In [12]:
df_fichas_com_votos.head(2) 

,uf_cand,nm_municipio,nm_cargo,nm_cand,sg_partido,ds_eleicao,ds_raca,ds_genero,nu_titulo_eleitor,dt_nascimento,ds_composicao,li_foto,id_municipio,votos,uf,municipio,capital,no_regiao_brasil
0,AC,ACRE,Governador,JORGE NEY VIANA MACEDO NEVES,PT,Não eleito,PARDA,MASC.,000780342402,1959-09-20,**,https://datarepo.psdb.org.br/fotos/10001616169...,1007,1670,AC,BUJARI,False,Norte
1,AC,ACRE,Governador,CYLMARA FERNANDES DA ROCHA GRIPP,MDB,Não eleito,PARDA,FEM.,002324422437,1973-09-04,REPUBLICANOS / MDB / PRTB / PL,https://datarepo.psdb.org.br/fotos/10001639519...,1007,946,AC,BUJARI,False,Norte


In [13]:
df_fichas_com_votos.columns

Index(['uf_cand', 'nm_municipio', 'nm_cargo', 'nm_cand', 'sg_partido',
       'ds_eleicao', 'ds_raca', 'ds_genero', 'nu_titulo_eleitor',
       'dt_nascimento', 'ds_composicao', 'li_foto', 'id_municipio', 'votos',
       'uf', 'municipio', 'capital', 'no_regiao_brasil'],
      dtype='object')

In [14]:
df_fichas_com_votos_estado = (
    df_fichas_com_votos.groupby(['uf_cand', 'nm_municipio', 'nm_cargo', 'nm_cand', 'sg_partido',
       'ds_eleicao', 'ds_raca', 'ds_genero', 'nu_titulo_eleitor',
       'dt_nascimento', 'ds_composicao', 'li_foto'])['votos']
    .sum()
    .reset_index()
)

In [15]:
df_fichas_com_votos_estado.head(5)

,uf_cand,nm_municipio,nm_cargo,nm_cand,sg_partido,ds_eleicao,ds_raca,ds_genero,nu_titulo_eleitor,dt_nascimento,ds_composicao,li_foto,votos
0,AC,ACRE,Deputado Estadual,ADALICIO ALVES FIGUEIREDO JUNIOR,PSB,Não eleito,PRETA,MASC.,006121262380,1973-08-26,**,https://datarepo.psdb.org.br/fotos/10001643318...,220
1,AC,ACRE,Deputado Estadual,ADEKSON DE PINHO LIMA,PODE,Não eleito,PARDA,MASC.,002064072437,1973-12-03,**,https://datarepo.psdb.org.br/fotos/10001650027...,77
2,AC,ACRE,Deputado Estadual,ADENILSON DUTRA DE SOUZA,CIDADANIA,Não eleito,PARDA,MASC.,005465192429,1986-12-12,**,https://datarepo.psdb.org.br/fotos/10001723797...,83
3,AC,ACRE,Deputado Estadual,ADERLANDIO MEDEIROS DA SILVA,PSOL,Não eleito,PRETA,MASC.,003469152488,1979-02-28,**,https://datarepo.psdb.org.br/fotos/10001595349...,623
4,AC,ACRE,Deputado Estadual,ADERLÂNDIO NASCIMENTO DE FRANÇA,PDT,Não eleito,PARDA,MASC.,003494752461,1978-12-17,**,https://datarepo.psdb.org.br/fotos/10001622765...,3555


In [16]:
df_fichas_com_votos_estado.shape[0]

25460

In [ ]:
#df_fichas_com_votos_estado.to_excel('/home/filipe/scripts/psdb-projetos/dashStreamlit/datasets/2022/teste_votos 2.xlsx', index=False)

In [ ]:

#df_fichas.loc[df_fichas['ds_sit_candidato'] != 'Deferido']
#df_federal = df_federal.query("sg_uf not in ['ZZ', 'VT']")